In [8]:
from utils.utils import build_market_image, prepare_forecast_data
# from environments.live_environments import LiveTradingEnv
from utils.clients import OandaClient
from Keys import *
# from utils.mappings import oanda_time_map
from utils.reward_functions import sharpe_reward_function
import pandas as pd
from json import dumps

import time
import datetime



In [9]:
# from neuralforecast.core import NeuralForecast
# forecasting_model=NeuralForecast.load('MultiHeadForecastingModel/')
# agent_dir='Agent/final_checkpoints/policies/default_policy'


In [10]:

from configs import defaults

base_asset=defaults.base_asset
quote_asset=defaults.quote_asset
test_net=True
time_frame=defaults.time_frame
positions=defaults.env_config['positions']

product_type=defaults.product_type

exchange=defaults.exchange
trade_target='/'.join([base_asset,quote_asset]) 
agent_path=defaults.agent_path
coin_pairs=defaults.COIN_PAIRS
trade_target

'USD/JPY'

In [11]:
client=OandaClient(api_key=oanda_api_key,
                    account_id=oanda_account_id,
                    paper=test_net,
                    symbol=trade_target,
                    time_frame=time_frame,
                    product_type=product_type

                    )
client.trade_client,client.base_asset,client.quote_asset,client.symbol,client.time_frame,client.oanda_time_frame



(<oandapyV20.oandapyV20.API at 0x12cc6c220>,
 'USD',
 'JPY',
 'USD_JPY',
 '1h',
 'H1')

In [12]:
from oandapyV20.endpoints import accounts, orders, pricing, instruments,transactions
import re

In [13]:
account=client.update_account()
account

,balance,pl,unrealizedPL,NAV,marginUsed,marginAvailable,positionValue
USD,9997.2558,-2.4401,0.0000,9997.2558,0.0000,9997.2558,0.0000


In [16]:
client.get_forex_position()

0

In [15]:
for symbol in coin_pairs:
    print(symbol)
    data=client.get_historical_data(symbol=symbol,start_date=datetime.datetime(2024,1,1))
    display(data.head(5))
    pair_name=re.sub(r'/|-|_','',symbol)
    data.to_pickle(f'data_forex/oanda-{pair_name}-{time_frame}.pkl')



EUR_USD


,date_close,open,high,low,close,volume,symbol
0,2024-01-01T22:00:00,1.10440,1.10453,1.10430,1.10450,188,EUR_USD
1,2024-01-01T23:00:00,1.10446,1.10446,1.10356,1.10368,1018,EUR_USD
2,2024-01-02T00:00:00,1.10367,1.10386,1.10342,1.10384,1328,EUR_USD
3,2024-01-02T01:00:00,1.10380,1.10434,1.10342,1.10350,2435,EUR_USD
4,2024-01-02T02:00:00,1.10351,1.10359,1.10184,1.10190,1968,EUR_USD


USD_JPY


,date_close,open,high,low,close,volume,symbol
0,2024-01-01T22:00:00,140.858,140.993,140.858,140.902,388,USD_JPY
1,2024-01-01T23:00:00,140.902,141.132,140.811,141.116,3558,USD_JPY
2,2024-01-02T00:00:00,141.115,141.384,141.100,141.142,3790,USD_JPY
3,2024-01-02T01:00:00,141.146,141.458,141.142,141.440,4419,USD_JPY
4,2024-01-02T02:00:00,141.441,141.673,141.366,141.565,3846,USD_JPY


GBP_USD


...

,date_close,open,high,low,close,volume,symbol
0,2024-01-01T22:00:00,1.32380,1.32563,1.32343,1.32449,306,USD_CAD
1,2024-01-01T23:00:00,1.32458,1.32484,1.32409,1.32448,873,USD_CAD
2,2024-01-02T00:00:00,1.32446,1.32478,1.32430,1.32451,913,USD_CAD
3,2024-01-02T01:00:00,1.32454,1.32454,1.32289,1.32420,2704,USD_CAD
4,2024-01-02T02:00:00,1.32410,1.32552,1.32395,1.32541,1479,USD_CAD


USD_CHF


,date_close,open,high,low,close,volume,symbol
0,2024-01-01T22:00:00,0.84052,0.84071,0.83969,0.84021,21,USD_CHF
1,2024-01-01T23:00:00,0.84033,0.84238,0.84033,0.84198,1294,USD_CHF
2,2024-01-02T00:00:00,0.84200,0.84264,0.84176,0.84255,1113,USD_CHF
3,2024-01-02T01:00:00,0.84254,0.84266,0.84192,0.84264,1278,USD_CHF
4,2024-01-02T02:00:00,0.84262,0.84477,0.84256,0.84476,1657,USD_CHF


NZD_USD


,date_close,open,high,low,close,volume,symbol
0,2024-01-01T18:00:00,0.63200,0.63259,0.63200,0.63259,2,NZD_USD
1,2024-01-01T21:00:00,0.63234,0.63273,0.63231,0.63273,11,NZD_USD
2,2024-01-01T22:00:00,0.63248,0.63290,0.63180,0.63186,433,NZD_USD
3,2024-01-01T23:00:00,0.63183,0.63228,0.63132,0.63145,1029,NZD_USD
4,2024-01-02T00:00:00,0.63146,0.63194,0.63118,0.63142,1131,NZD_USD


In [17]:
params={"instruments":client.symbol}

req=instruments.Instrument(client.account_id, params=params)
trade_info=client.trade_client.request(req)
trade_info.keys()
pricing


AttributeError: module 'oandapyV20.endpoints.instruments' has no attribute 'Instrument'

In [ ]:
strip_symbol=r'/|_|-|USD'
symbol='USD-JPY'
base_asset= re.sub(strip_symbol,'',symbol)
base_asset


In [37]:
def strip_quote(symbol,quote_asset):
    if symbol==quote_asset:
        return symbol
    else:
        strip_pattern=f'/|_|-|{quote_asset}$'
        asset=re.sub(strip_pattern,'',symbol)
        return asset



In [ ]:
[strip_quote(symbol,client.base_asset) for symbol in client._account.index]

In [ ]:

account_frame=pd.DataFrame([account])

account_frame.rename(columns={'currency':'symbol'},inplace=True)
account_frame.set_index('symbol',inplace=True)

account_frame=account_frame[['balance','pl','unrealizedPL','NAV','marginUsed','marginAvailable','positionValue']]

account_frame

In [ ]:
trade_frame=pd.DataFrame(trades)
trade_frame.rename(columns={'instrument':'symbol','currentUnits':'balance'},inplace=True)
trade_frame.set_index('symbol',inplace=True)
trade_frame




In [ ]:
full_account_frame=pd.concat([account_frame,trade_frame],axis=0)

full_account_frame

In [ ]:
positions=client.trade_client.list_futures_positions()
positions=pd.DataFrame(positions['positions'])
positions=positions.set_index('product_id')
positions

In [ ]:
positions

In [ ]:

product_frame=pd.DataFrame(products['products'])
product_frame=product_frame[product_frame['view_only']==False]

# product_frame=product_frame[product_frame['trading_disabled']==False]

product_frame

In [ ]:
trade_info

In [ ]:
# products=client.trade_client.get_products()
now=datetime.datetime.now()
start_time=now-datetime.timedelta(hours=input_size*2)

int(now.timestamp())

In [ ]:
fills=client.get_orders()
fills

In [ ]:
time.sleep(10)

In [ ]:
client.sell('BTC-USDC',base_size='.000014')

In [ ]:
time.sleep(10)

In [ ]:
fills=client.get_orders()
fills